<a href="https://colab.research.google.com/github/diputs03/AI-Studies/blob/main/Creating_network/dymamic_architect_rebuilt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
#@title Aiming a Dynaimic Graph-structured NeuronNetwork
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sympy

In [102]:
"""
Activation function in this case in tanh
Loss is the Euclidean loss
"""
class Model:
  class Neuron:
    def __init__(self, name, prev, next):
      self.name, self.prev, self.next = name, prev, next
      self.bias = 0
  def __init__(self, input_size, output_size):
    self.input_size = input_size
    self.output_size = output_size
    self.Input_layer = [Model.Neuron(f"input{i}", [], []) for i in range(input_size)]
    self.Output_layer = [Model.Neuron(f"output{o}", [], []) for o in range(output_size)]
    for i in range(input_size): self.Input_layer[i].next = self.Output_layer
    for o in range(output_size): self.Output_layer[o].prev = self.Input_layer
    self.weight = {(u,v): 1 for u in self.Input_layer for v in self.Output_layer}
    self.all_neurons = self.Input_layer + self.Output_layer

  def forward(self, X, batch_size):
    assert X.shape == (batch_size,self.input_size)
    a = {q: [0 for _ in range(batch_size)] for q in self.all_neurons}
    for i in range(self.input_size):
      a[self.Input_layer[i]] = X.T[i].copy()
    stack = self.Input_layer.copy()
    cnt = {q: 0 for q in self.all_neurons}

    while len(stack) != 0:
      c = stack.pop()
      a[c] = np.tanh(a[c] + c.bias)
      for n in c.next:
        a[n] = a[n] + a[c]*self.weight[(c,n)]
        cnt[n] += 1
        if cnt[n] == len(n.prev):
          stack.append(n)
    return a

  def eval(self, X, batch_size=len(X)):
    a = self.forward(X, batch_size)
    return np.array([a[o] for o in self.Output_layer]).T

  def backward(self, X, Y, batch_size):
    assert X.shape == (batch_size,self.input_size)
    assert Y.shape == (batch_size,self.output_size)
    a = self.forward(self, X, batch_size)
    par_a = {q: [0 for _ in range(batch_size)] for q in self.all_neurons}
    for o in range(self.output_size):
      par_a[self.Output_layer[o]] = 2 * (a[self.Output_layer[O]] - Y.T[O])
    stack = self.Output_layer.copy()
    cnt = {q: 0 for q in self.all_neurons}

    while len(stack) != 0:
      c = stack.pop()
      par_b = par_a[c] * (1-a[c]**2)
      for p in c.prev:
        par_a[p] += par_a[c] * (1-a[c]**2) * self.weight[(p,c)]
        par_pc = par_a[c] * (1-a[c]**2) * a[p]
        cnt[p] += 1
        if cnt[p] == len(p.next):
          stack.append(p)

In [97]:
mod = Model(2, 2)
X = np.array([[0,0], [0,1], [1,0], [1,1]])
print(mod.eval(X))

4
[[0.         0.        ]
 [0.64201499 0.64201499]
 [0.64201499 0.64201499]
 [0.90925167 0.90925167]]
